**IMPORTAÇÃO DAS BIBLIOTECAS**

In [ ]:
# Importando as bibliotecas utilizadas e métodos ou atributos utilizados

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

**IMPORTAÇÃO DOS DADOS EM CSV**

In [ ]:
# Conectando o Drive para acessar as tabelas disponibilizadas

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Atribuindo as planilhas a seu respectivo dataframe


df_demo = pd.read_csv('/content/drive/MyDrive/COLAB COPIAS/testes projeto/BDIPMamaV11-INTELIDemograficosTt_DATA_LABELS_2023-01-24_1922.csv')
df_hist = pd.read_csv('/content/drive/MyDrive/COLAB COPIAS/testes projeto/BDIPMamaV11-INTELIHistopatologia_DATA_LABELS_2023-01-24_1924.csv')
df_peso = pd.read_csv('/content/drive/MyDrive/COLAB COPIAS/testes projeto/BDIPMamaV11-INTELIPesoEAltura_DATA_LABELS_2023-01-24_1926.csv')
df_tumor = pd.read_csv('/content/drive/MyDrive/COLAB COPIAS/testes projeto/BDIPMamaV11-INTELIRegistroDeTumo_DATA_LABELS_2023-01-24_1924.csv')


**SELEÇÃO PRIMÁRIA DE TABELAS**  

In [ ]:
# Após debate inicial do grupo, decidiu-se seguir com as seguintes colunas em cada dataframe

df_demo = df_demo[['Record ID', 'Escolaridade', 'Idade do paciente ao primeiro diagnóstico', 'Sexo', 'Data da última informação sobre o paciente', 'Última informação do paciente', 'Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]', 'Já ficou grávida?', 'Idade na primeira gestação', 'Amamentou na primeira gestação?', 'Por quanto tempo amamentou?', 'Idade da primeira mentruação', 'Já fez uso de drogas?',	'Atividade Física',	'Consumo de tabaco',	'Consumo de álcool', 'Possui histórico familiar de câncer?', 'Regime de Tratamento']]


df_peso = df_peso[['Record ID','Peso', 'Altura (em centimetros)', 'IMC']]


df_tumor = df_tumor[['Record ID', 'Código da Topografia (CID-O)', 'Código da Morfologia de acordo com o CID-O', 'Estadio Clínico  ', 'Grupo de Estadio Clínico', 'Classificação TNM Clínico - T', 'Classificação TNM Clínico - N', 'Classificação TNM Clínico - M', 'Combinação dos Tratamentos Realizados no Hospital', 'Lateralidade do tumor', 'Tempo desde o diagnóstico até a primeira recidiva  ', 'Descrição da Morfologia de acordo com o CID-O  (CID-O - 3ª edição)', 'Descrição da Topografia ', 'Com recidiva à distância', 'Com recidiva regional', 'Com recidiva local']]


df_hist = df_hist.drop(columns='Indice H (Receptorde progesterona)')



***LIMPEZA DE LINHAS TOTALMENTE VAZIAS OU POBRES DE CONTEÚDO***

In [ ]:
#não foi necessário fazer esse processo com o df_demo

#A lógica utilizada foi manter nos dataframes apenas as linhas que tenham pelo menos uma célula não nula, eliminando as linhas vazias

df_tumor = df_tumor[df_tumor.isnull().sum(axis=1) < 14]

df_peso = df_peso[df_peso.isnull().sum(axis=1) < 3]

df_hist = df_hist[df_hist.isnull().sum(axis=1) < 10]

print(df_tumor.shape)
print(df_peso.shape)
print(df_hist.shape)
print(df_demo.shape)
print(df_peso['Record ID'].value_counts())

*TRATAMENTO DA TABELA **PESO** PARA EVITAR REPETIÇÕES* // TRATAMENTO DE OUTLIERS

In [ ]:
#SETANDO O NUMERO DE CASAS DECIMAIS
pd.set_option('display.precision',1)

menores = df_peso[(df_peso.IMC > 5)]
maiores = menores[(menores.IMC < 50)]


#Agrupando por ID e colocando a média 
df_peso = maiores[(maiores.IMC != np.inf)].groupby(['Record ID'], as_index=False).mean()

#pegar só a ultima ocorrencia

RETIRANDO LINHAS COM MAIS DE UMA OCORRÊNCIA, **MANTENDO O ÚLTIMO REGISTRO**

In [ ]:
#Aqui utilizou-se o comando drop_duplicates com o parâmetro keep = last
#de forma que o dataframe retire todas as linhas que tenham o mesmo Record ID
#mantendo apenas a última linha na tabela, a fim de manter o agrupamento

df_hist = df_hist.drop_duplicates(subset=['Record ID'], keep='last')
df_tumor = df_tumor.drop_duplicates(subset=['Record ID'], keep='last')

print(df_peso['Record ID'].nunique())
print(df_hist['Record ID'].nunique())
print(df_tumor['Record ID'].nunique())
print(df_demo['Record ID'].nunique())

print('-----------------')

print(df_peso['Record ID'].value_counts().sum())
print(df_hist['Record ID'].value_counts().sum())
print(df_tumor['Record ID'].value_counts().sum())
print(df_demo['Record ID'].value_counts().sum())

**MERGE DAS TABELAS**

In [ ]:
#Fazendo o merge das tabelas com o comando merge()

df_merge = df_demo.merge(df_tumor)
df_merge = df_merge.merge(df_hist)
df_merge = df_merge.merge(df_peso)

df_dados = df_merge
df_dados.isnull().sum()



**SELEÇÃO DE COLUNAS ESPECÍFICAS PARA TRABALHAR**

In [ ]:
#Após discussão da equipe, com base nos materiais que nos foram fornecidos para estudo do problema
#usando guidelines, entrevistas e artigos como referência
#decidimos utilizar as seguintes colunas no nosso modelo

dfz = df_dados[['Record ID', 'Idade do paciente ao primeiro diagnóstico', 'Última informação do paciente', 'Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]', 'Já ficou grávida?', 'Regime de Tratamento', 'Classificação TNM Clínico - M', 'Classificação TNM Clínico - N', 'Classificação TNM Clínico - T', 'Lateralidade do tumor', 'Com recidiva à distância', 'Com recidiva regional', 'Com recidiva local', 'Estadio Clínico  ', 'Combinação dos Tratamentos Realizados no Hospital']]
for coluna in dfz:
  print(f"{df_dados[coluna].unique()}---")



TRATAMENTO DE MISSINGS DAS COLUNAS SELECIONADAS

In [ ]:
#SETANDO O NUMERO DE CASAS DECIMAIS
pd.set_option('display.precision',0)

#Cria-se duas variações do dataframe utilizado
#Sendo elas respectivamente para fazer a codificação e normalização da coluna e retirar os outliers, afim de que a média calculada seja realista
tes = dfz.dropna(subset=['Última informação do paciente'])
tes2 = dfz.dropna(subset=['Última informação do paciente'])


tes2['Idade do paciente ao primeiro diagnóstico'].fillna(tes['Idade do paciente ao primeiro diagnóstico'].describe()['mean'], inplace= True)
#Aqui escolhemos substituir a idade com média temporariamente para poder fazer a analise dos dados e depois modificamos novamente para "0", para facilitar o reconhecimento de dados ausentes na hora da codificação
tes2['Já ficou grávida?'].fillna("Não Informado Gravida", inplace= True)
tes2['Regime de Tratamento'].fillna("Não Informado Tratamento", inplace= True)


tes['Idade do paciente ao primeiro diagnóstico'].fillna("0", inplace= True)
tes['Já ficou grávida?'].fillna("Não Informado Gravida", inplace= True)
tes['Regime de Tratamento'].fillna("Não Informado Tratamento", inplace= True)

#EMBORA TENHA SIDO UTILIZADO DF2 PARA ANALISES, A CONTINUIDADE DO CÓDIGO SE DARÁ COM O DF1 PORQUE ELE ESTÁ COM A IDADE ADAPTADA PARA SER CODIFICADA


**TRATAMENTO DE ALGUNS OUTLIERS DAS COLUNAS SELECIONADAS**

In [ ]:
#ELIMINANDO PACIENTES ACIMA DE 90 ANOS PORQUE POSSUIMOS POUCAS AMOSTRAGENS ACIMA DESSA FAIXA ETÁRIA E PARA EVITAR UNDERFITTING
index_idade = tes2[ (tes2['Idade do paciente ao primeiro diagnóstico'] >= 90)].index
tes2.drop(index_idade, inplace = True)
tes.drop(index_idade, inplace = True)

#ELIMINANDO PACIENTES QUE POSSUEM REGISTRO DE TUMOR ABAIXO DE 20 DIAS PORQUE ISSO É MUITO POUCO E PROVAVELMENTE A DATA ESTÁ BUGADA
index_diastumor = tes2[ (tes2['Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]'] <= 20)].index
tes2.drop(index_diastumor, inplace = True)
tes.drop(index_diastumor, inplace = True)


**RECONHECIMENTO DE COLUNAS NUMÉRICAS X COLUNAS CATEGÓRICAS**

In [ ]:
#Através do comando dtypes, identificamos quais são as tabelas categóricas e numéricas

cont = 0
listacolcat = []
for coluna in tes2.dtypes:
  if coluna == 'float64' or coluna == 'int64':
    tipo = "Coluna Númerica"

  elif coluna == object:
      tipo = "Coluna Categórica"
      listacolcat.append(tes2.columns[cont])
      
  print(f'{tes2.columns[cont]} é {tipo}\n========\n')
  cont+=1


  **ESTATÍSTICA DESCRITIVA DAS COLUNAS NÚMERICAS**
  

In [ ]:
#Para as colunas numéricas, usamos o comando describe() para realizar a estatística descritiva


tes2.describe()


**ESTATÍSTICA DESCRITIVA DAS COLUNAS CATEGÓRICAS**

In [ ]:
#Para as colunas categóricas, usamos o comando value_counts() para realizar a estatística descritiva

for coluna in listacolcat:
  print(f'Coluna:{coluna}\n\n{tes2[coluna].value_counts()}\n\n\n') 

**3 GRÁFICOS RELACIONAIS ENTRE VARIÁVEIS ESCOLHIDAS PELO GRUPO**


Aqui relacionamos a idade do paciente e o tempo desde o último tumor

In [ ]:
#Gráfico gerado através do lineplot
sns.lineplot(data=tes2, x='Idade do paciente ao primeiro diagnóstico', y='Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]')



Nesse outro gráfico relacionamos a situação do paciente com sua idade e explicitamos o regime de tratamento utilizado de cada um

In [ ]:
#Gráfico gerado através do barplot

sns.barplot(data=tes2, x="Última informação do paciente", y="Idade do paciente ao primeiro diagnóstico", hue="Regime de Tratamento")

Por último, nesse gráfico tentamos enxergar a relação entre a lateralidade do tumor e o tempo desde o último tumor, e novamente tentamos explicitar qual foi o regime de tratamento utilizado

In [ ]:
#Gráfico gerado através do catplot


sns.catplot(data=tes2, x="Lateralidade do tumor", y="Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]", hue='Regime de Tratamento', aspect=10.7/7.3, height=10)

# **CODIFICAÇÃO **

CODIFICAÇÃO IDADE (label) E ÚLTIMA INFORMAÇÃO (one-hot)

In [ ]:
#SETANDO O NUMERO DE CASAS DECIMAIS

pd.set_option('display.precision',1)

#transformando a coluna a seguir em inteiro
tes['Idade do paciente ao primeiro diagnóstico'] = tes['Idade do paciente ao primeiro diagnóstico'].astype('int64')

#dividindo as idades em intervalos, de forma que cada pessoa esteja em grupos com base nos intervalos estipulados
tes['Idade COD'] = pd.cut(tes['Idade do paciente ao primeiro diagnóstico'], bins=[-1,1,30,45,60,89], labels=['-1','1','2','3','4'])
print(tes.shape)
#-1 -> NÃO INFORMADO
# 1 = Idade entre 1~30 anos
# 2 = Idade entre 30~45 anos
# 3 = Idade entre 45~60 anos
# 4 = Idade 60+ anos


In [ ]:
#Aplicação de lógica de OneHotEncoder para a seguinte coluna

ohe = OneHotEncoder()
opa = ohe.fit_transform(tes['Última informação do paciente'].values.reshape(-1,1)).toarray()

dfohe = pd.DataFrame(opa, columns=['Óbito por Câncer','Vivo SOE', 'Vivo com Câncer', 'Óbito por outras causas SOE'])
dfohe.shape
df = pd.concat([tes,dfohe], axis=1)
df

CODIFICAÇÃO TEMPO DE SEGUIMENTO (LABEL)

In [ ]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita
df = df.drop(df[df['Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]'] == 0].index)

df['Tempo ultimo tumor'] = pd.cut(df['Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]'], bins=[0,365,730,1825,2920,4000], labels=['1','2','3','4','5'])


df

CODIFICAÇÃO "JÁ FICOU GRÁVIDA" (ONE-HOT)

In [ ]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

df['Já ficou grávida?'].fillna("Não Informado Gravida", inplace= True)
ohegravida = ohe.fit_transform(df['Já ficou grávida?'].values.reshape(-1,1)).toarray()

df_ohegravida = pd.DataFrame(ohegravida, columns=['Não Grávida','Não informado Grávida','Grávida'])

df = pd.concat([df,df_ohegravida], axis=1)

CODIFICAÇÃO "REGIME DE TRATAMENTO" (ONE-HOT)

In [ ]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

df['Regime de Tratamento'].fillna("Não Informado Tratamento", inplace= True)
ohetrat = ohe.fit_transform(df['Regime de Tratamento'].values.reshape(-1,1)).toarray()


df_ohetrat = pd.DataFrame(ohetrat, columns=['Tratamento Não Informado','Não fez Quimioterapia','Paliativo','Terapia Adjuvante','Terapia Neoadjuvante'])
df = pd.concat([df,df_ohetrat], axis=1)



CODIFICAÇÃO "LATERALIDADE DO TUMOR"

In [ ]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

df['Lateralidade do tumor'].fillna("Não Informado Lateralidade", inplace= True)
ohelateral = ohe.fit_transform(df['Lateralidade do tumor'].values.reshape(-1,1)).toarray()


df_ohelat = pd.DataFrame(ohelateral, columns=['Bilateral','Direita','Esquerda','Lateralidade Não informada','Lateralidade não se aplica'])

df = pd.concat([df,df_ohelat], axis=1)
df

CODIFICAÇÃO DAS RECIDIVAS (BINARY)

In [ ]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

df['Com recidiva local'].fillna("Não", inplace= True)
df['Recidiva Local'] = df['Com recidiva local']
df["Recidiva Local"] = np.where(df["Com recidiva local"] == "Não", 0, 1)

df['Com recidiva regional'].fillna("Não", inplace= True)
df['Recidiva Regional'] = df['Com recidiva regional']
df["Recidiva Regional"] = np.where(df["Com recidiva regional"] == "Não", 0, 1)

df['Com recidiva à distância'].fillna("Não", inplace= True)
df['Recidiva à distância'] = df['Com recidiva à distância']
df["Recidiva à distância"] = np.where(df["Com recidiva à distância"] == "Não", 0, 1)

df

CODIFICAÇÃO "ESTADIO CLINICO" (LABEL)

In [ ]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

label_encoder = LabelEncoder()
df['Estadio Clínico  '].fillna("IV", inplace= True)

df['Estadio Clínico COD'] = label_encoder.fit_transform(df['Estadio Clínico  '])
df['Estadio Clínico  '].unique()
df['Estadio Clínico COD'].value_counts()
# 0 - 0
# I - 1
# IA - 2
# IB - 3
# IB1 - 4
# IB2- 5
# II - 6
# IIA - 7
# IIB - 8
# IIC- 9
# III- 10
# IIIA - 11
# IIIB - 12
# IIIC - 13
# IV - 14
# IVA - 15
# IVB - 16
# IVC - 17
# Y: NA - 18
# X : não foi possivel determinar - 19

CODIFICAÇÃO "COMBINAÇÃO DE TRATAMENTOS" (ONE HOT)

In [ ]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

df['Combinação dos Tratamentos Realizados no Hospital'].fillna("Nenhum tratamento", inplace= True)

ohecombtrat = ohe.fit_transform(df['Combinação dos Tratamentos Realizados no Hospital'].values.reshape(-1,1)).toarray()


df_combtrat = pd.DataFrame(ohecombtrat, columns=['C','C+Q','C+R+Q','C+R+Q+H','C+R','Nenhum Tratamento','Outras Combinações','Q','R','R+Q'])
# C = CIRURGIA
# Q = QUIMIOTERAPIA
# R = RADIOTERIA
# H = HORMONIOTERAPIA


df = pd.concat([df,df_combtrat], axis=1)

df

CODIFICAÇÃO TNM - M (LABEL)

In [ ]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

df['Classificação TNM Clínico - M'].fillna("X - nao foi possivel determinar", inplace= True)

df['Classificação TNM - M'] = label_encoder.fit_transform(df['Classificação TNM Clínico - M'])

df
#0 - 0
#1 - 1
#2 - 1A
#3 - 1B
#4 - X - nao foi possivel determinar
#5 - Y: na


df['Classificação TNM - M'].value_counts()

CODIFICAÇÃO TNM - N (LABEL)

In [ ]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

df['Classificação TNM Clínico - N'].fillna("X - nao foi possivel determinar", inplace= True)

df['Classificação TNM - N'] = label_encoder.fit_transform(df['Classificação TNM Clínico - N'])


# 0 - 0 
# 1 - 1
# 2 - 1A
# 3 - 1B
# 4 - 2
# 5 - 2A
# 6 - 2B
# 7 - 3
# 8 - 3A
# 9 - 3B
# 10 - 3C
# 11 - X -nao foi possivel determinar
# 12 - Y: Na


df.isnull().sum()


CODIFICAÇÃO TNM - T (LABEL)

In [ ]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita


df['Classificação TNM Clínico - T'].fillna("X - nao foi possivel determinar", inplace= True)

df['Classificação TNM - T'] = label_encoder.fit_transform(df['Classificação TNM Clínico - T'])


# 0 - 0
# 1 - 1
# 2 - 1A
# 3 - 1B
# 4 - 1C
# 5 - 1MIC
# 6 - 2
# 7 - 2A
# 8 - 2B
# 9 - 3
# 10 - 3A 
# 11 - 3B
# 12 - 4
# 13 - 4A
# 14 - 4B
# 15 - 4C
# 16 - 4D
# 17 - CDIS
# 18 - CLIS
# 19 - IS
# 20 - X - nao foi possivel determinar
# 21 - Y: Na
 
df['Classificação TNM - T'].value_counts()

TRATAMENTO DE MISSINGS OCORRIDOS DEVIDO A UM BUG DO ONEHOT ENCODED

E 

SEPARAÇÃO ENTRE DOIS DF's -> UM POSSUI SOMENTE AS COLUNAS CODIFICADAS E O OUTRO POSSUI TODAS

In [ ]:
#Correção de bugs observados

df.isnull().sum()
df.columns[15]

df_x = df.iloc[:,15:52]

df_x = df_x.dropna(subset=['Vivo com Câncer','Grávida', 'Idade COD', 'Tempo ultimo tumor'])

df_x.isnull().sum()

# **DESENVOLVIMENTO DE HIPÓTESES **

**1a Hipótese** -> Com base nos dados que possuímos e analisando-os graficamente, podemos analisar que pacientes na faixa etária de 40-60 que nunca ficaram grávidas possuem alto risco de morte.

In [ ]:
sns.set_theme(style="whitegrid")

#fig, axes = plt.subplots(1,2, sharex=True, figsize=(25,5))
sns.catplot(data=df,kind='violin',inner='stick',split=True ,x='Já ficou grávida?', y='Idade do paciente ao primeiro diagnóstico',hue='Óbito por Câncer', aspect=10.7/7.3, height=10)

**2a Hipótese** -> Para as pessoas do grupo de estadio clínico IV que estejam perto do 60 anos, o tratamento indicado é o tratamento adjuvante, já que podemos ver no gráfico, um pico de pessoas sobevivendo, muito acima do outro cenário (morte).

In [ ]:
#Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos [dt_pci]

tt = df[(df['Estadio Clínico COD'] == 14)]
sns.catplot(data=tt,kind='violin',inner='stick',split=True ,x='Regime de Tratamento', y='Idade do paciente ao primeiro diagnóstico',hue='Vivo SOE', aspect=10.7/7.3, height=10)

**3a Hipótese** ->  O tratamento adjuvante não é indicado para pacientes na faixa etária entre 45~60 que estão no estadio clínico IIA. Conforme podemos analisar no gráfico abaixo, há uma curva bastante acentuada para pessoas nessa idade.

In [ ]:
#sns.catplot(data=rr,kind='violin',inner='stick',split=True ,x='Regime de Tratamento', y='Idade do paciente ao primeiro diagnóstico',hue='Vivo SOE', aspect=10.7/7.3, height=10)

rr = df[(df['Estadio Clínico COD'] == 7)]
sns.catplot(data=rr,kind='violin',inner='stick',split=True ,x='Regime de Tratamento', y='Idade do paciente ao primeiro diagnóstico',hue='Óbito por Câncer', aspect=10.7/7.3, height=10)
#index_diastumor = tes2[ (tes2['Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]'] <= 20)].index
